In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import mapping
import folium
import gmaps
import os
import requests
import shutil

In [2]:
with open('../resources/key.txt') as f:
    token = f.read()
    
gmaps.configure(api_key = token)

In [7]:
input_folder = '../data/railways/mbta_rapid_transit/'
input_file = input_folder+'MBTA_ARC.shp'

output_folder = '../data/output_images/BostonMTBA/'
output_csv = output_folder+'BOS.csv'

update_folder = '../data/output_images/BostonMTBA/set_2/'

### Import Routes

In [6]:
routes = gpd.read_file(input_file)

routes = routes.to_crs({'init' :'epsg:4269'})

routes.head()

,LINE,ROUTE,GRADE,SHAPE_LEN,geometry
0,GREEN,C D,7,458.386096,LINESTRING (-71.10139485815532 42.347577953996...
1,GREEN,B C D E,7,2121.006147,LINESTRING (-71.07920620455907 42.349579702367...
2,RED,A - Ashmont B - Braintree C - Alewife,7,7955.708839,LINESTRING (-71.08197671857094 42.361992845118...
3,GREEN,C E,7,1241.873692,LINESTRING (-71.05945866821224 42.359296597842...
4,GREEN,B C D,7,1466.201312,LINESTRING (-71.09616476184985 42.348960933708...


In [5]:
routes.shape

(136, 5)

In [7]:
'''
Get all points
'''

points = []
for i in range(0,len(routes.index.tolist())):
#     print('Route',i)
    try:
        route = routes.iloc[i]
        g = [m for m in route.geometry]
#         print('\t',len(g),'sections')
        for j in range(0,len(g)):
#             print('   section',j)
            sec_points = mapping(g[j])["coordinates"]
            for k in range(0,len(sec_points)):
#                 print('      point',k)
                points.append(sec_points[k])
    except Exception as e:
        print("Skipped route number",i,'because',e)

print(len(points),'points')

Skipped route number 0 because 'LineString' object is not iterable
Skipped route number 1 because 'LineString' object is not iterable
Skipped route number 2 because 'LineString' object is not iterable
Skipped route number 3 because 'LineString' object is not iterable
Skipped route number 4 because 'LineString' object is not iterable
Skipped route number 5 because 'LineString' object is not iterable
Skipped route number 6 because 'LineString' object is not iterable
Skipped route number 7 because 'LineString' object is not iterable
Skipped route number 8 because 'LineString' object is not iterable
Skipped route number 9 because 'LineString' object is not iterable
Skipped route number 10 because 'LineString' object is not iterable
Skipped route number 11 because 'LineString' object is not iterable
Skipped route number 12 because 'LineString' object is not iterable
Skipped route number 13 because 'LineString' object is not iterable
Skipped route number 14 because 'LineString' object is not

In [ ]:
'''
Get points for 1 or more routes
'''

rows = [0]

points = []
for i in range(0,len(routes.OBJECTID.tolist())):
    if i in rows:
    #     print('Route',i)
        try:
            route = routes.iloc[i]
            g = [m for m in route.geometry]
    #         print('\t',len(g),'sections')
            for j in range(0,len(g)):
    #             print('   section',j)
                sec_points = mapping(g[j])["coordinates"]
                for k in range(0,len(sec_points)):
    #                 print('      point',k)
                    points.append(sec_points[k])
        except Exception as e:
            print("Skipped route number",i,'because',e)

print(len(points),'points')

In [ ]:
'''
Get points for single route/section
'''

row = 0
section = 0

points = []
try:
    route = routes.iloc[row]
    g = [i for i in route.geometry]
    len(g)

    points = mapping(g[section])["coordinates"]
except Exception as e:
    print("Skipped route number",row,'because',e)
    
print(len(points),'points')

In [8]:
'''
Get subselection of points
'''

route_points = []
for point in range(0,len(points),7):
    tmp = points[point]
    route_points.append(tmp)

print(len(route_points))

51


In [9]:
'''
Create dataframe
'''

df = pd.DataFrame(route_points, columns=['Longitude','Latitude'])

tmp = list(df.Latitude.tolist())
names = []
for i in range(0,len(tmp)):
    name = str(df.iloc[i].Longitude)+'_'+str(df.iloc[i].Latitude)
    names.append(name)

df['Name'] = names

columns = ['Name','Longitude','Latitude']
df= df[columns]

df.head()

,Name,Longitude,Latitude
0,-71.06445861625593_42.34737609306864,-71.064459,42.347376
1,-71.0641837596865_42.34846784549941,-71.064184,42.348468
2,-71.06492663944647_42.34579589916245,-71.064927,42.345796
3,-71.06492663944647_42.34579589916245,-71.064927,42.345796
4,-71.06876179011799_42.341708782139065,-71.068762,42.341709


In [10]:
'''
Export points
'''

df.to_csv(output_csv,header=True,index=None)

In [8]:
'''
Open csv
'''

df = pd.read_csv(output_csv)
df.head()

,Name,Longitude,Latitude,Catenary
0,-70.99141024363398_42.39987983654313,-70.991410,42.399880,1
1,-70.9916645142435_42.41343348702443,-70.991665,42.413433,1
2,-70.99169316827988_42.3987030963677,-70.991693,42.398703,1
3,-70.99195436847552_42.40432141901854,-70.991954,42.404321,1
4,-70.99217754599157_42.39778478905404,-70.992178,42.397785,1


In [5]:
'''
Plot markers
'''

rows = df.index.tolist()

marker_points = []
for row in rows:
    marker_points.append((df.iloc[row].Latitude,df.iloc[row].Longitude))
    
# Set map centerpoint
coords = marker_points[0]
       
# Define map
m = folium.Map(
    location = coords,
    zoom_start = 13
)
  
# Add points    
for mp in marker_points:
    folium.Marker(mp).add_to(m)
m

In [108]:
'''
Get satellite preview for image
'''

row = 0

figure_layout = {
    'width': '100%',
    'height': '800px'
}
fig = gmaps.figure(center = (df.iloc[row].Latitude,df.iloc[row].Longitude), zoom_level = 19,map_type='SATELLITE',layout=figure_layout)
fig

Figure(layout=FigureLayout(height='800px', width='100%'))

In [9]:
'''
Get all images 
'''

root = os.path.dirname(os.path.abspath('Africa.ipynb'))
image_folder = root[:-9]+update_folder[3:]
image_folder

url = 'https://maps.googleapis.com/maps/api/staticmap?'
rows = df.index.tolist()
for i in range(0,len(rows)):
    row = df.iloc[i]
    center = str(row.Latitude)+','+str(row.Longitude)
    payload = {
        'center': center, 
        'zoom': 20,
        'size': '640x640',
        'scale': 2,
        'format': 'png32',
        'maptype': 'satellite',
        'key': token
    }
    
    r = requests.get(url,params=payload,stream=True)

    name = df.iloc[i]['Name']
    filename = image_folder+name+'.png'
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f) 